In [ ]:
%config Completer.use_jedi = False
import os
import re
import pandas
import numpy
import sqlite3
import graphviz
from graphviz import Digraph

In [ ]:
def main(inpfile,graphformat="pdf"):
    basename,_=os.path.splitext(os.path.basename(inpfile))
    dbfile=f"output/{basename}.db"
    setup_db(dbfile)

    txt = gobble_file(inpfile)
    parse_txt(txt, dbfile)
    
    xlfile=f"output/{basename}.xlsx"
    pandas.DataFrame(["hello"]).to_excel(xlfile)
    #save_table_to_excel(dbfile,"PROCESS-LOAD",xlfile)
    #save_table_to_excel(dbfile,"SPACE",xlfile)
    for cmd in ['SUCTION-GROUP',
            'REFG-SYSTEM',
            'REFG-CIRCUIT',
            'CONDENSER',
            'COMPRESSOR',
            'EQUIP-CTRL',
            'LOAD-MANAGEMENT',
            'SUBCOOLER',
            'DESUPERHEATER',
            'SYSTEM',
            'ZONE',
            'PROCESS-LOAD']:
        save_table_to_excel(dbfile,cmd,xlfile)
    
    pdf_file=f'output/{basename}.gv'
    edges = get_edges(dbfile)
    nodes = get_nodes(dbfile)
    cmd2shape = get_cmd2shape(dbfile)
    dot=make_graph(basename,nodes,edges,cmd2shape,pdf_file,graphformat)
    
    return dot

# Read in the entire INP file.
def gobble_file(inpfile):
    txt=""
    with open(inpfile) as f:
        for line in f:
            # Enforce DOE2's line limit 80 characters
            l = line[:80]
            # Strip comments. This is the old fashioned "line construction" method.
            l = re.sub(r"\$.*$","",re.sub(r"\$.*?\$","",l)).strip()
            if len(l) > 0:
                txt += l + "\n"
    return txt

# A very basic BDL parser.
def parse_txt(txt,dbfile):
    command_list=[]
    arg_list=[]

    # Loop through complete command blocks e.g.
    # [UNAME = ] COMMAND 
    #   keyword1 = value
    #   leftovers
    # ..
    for i,m1 in enumerate(re.finditer(r"\s*(.*?)\s*\.\.\s*", txt, re.DOTALL)):
        g1=m1.group(1)
        
        # From the first line, find out if the command has a U-NAME.
        m2=re.match(r"(?P<uname>.*?)\s*=\s*(?P<tcmd>.*)|(?P<icmd>.*)",g1)
        kind = m2.lastgroup
        cmd,uname=None,None
        if kind == 'icmd':
            # intransitive verb
            cmd=m2.group('icmd')
        elif kind == 'tcmd':
            # intransitive verb
            cmd=m2.group('tcmd')
            uname0=m2.group('uname')
            # If the U-NAME is quoted, unquote it.
            mu0=re.match(r'"(?P<quoted>.*)"|(?P<unquoted>.*)',uname0)
            uname=mu0.group(mu0.lastgroup)
        if len(cmd) == 0:
            cmd = "$noop"
        command_list.append((i,cmd,uname))
        tail2=g1[m2.end():]

        # Loop through lines after the first line.
        # \s*                                            whitespace
        # (.+?)\s*                                       keyword
        # =\s*                                           =
        # (                                              either
        #  \((?:(?:\s*"[^"]+?"|\s*[^\s\)]+?)[,\s]*)+\)   (list1, "of2", non-list3 items4,)
        # |                                              or
        #  (?:.*)                                        item
        # )\s*                                           followed by whitespace
        j,m3=-1,None
        for j,m3 in enumerate(re.finditer(r'\s*(.+?)\s*=\s*(\((?:(?:\s*"[^"]+?"|\s*[^\s\)]+?)[,\s]*)+\)|(?:.*))\s*', tail2)):
            key,val=m3.group(1),m3.group(2)
            arg_list.append((i,j,key,val))
        
        tail3 = tail2[m3.end():] if j>=0 else tail2
        if len(tail3) > 0:
            arg_list.append((i,j+1,"$tail",tail3))
        # Some commands always have "leftover" lines that are not of the form a = b.
        # If there are leftover lines but not expected, emit a warning.
        ignore_tail = (len(tail3) == 0) \
            or cmd.startswith("RUN-PERIOD") \
            or cmd.startswith("SCHEDULE") \
            or tail3.startswith("COEF ")
        if not ignore_tail:
            print(f"[{basename}] Warning: Unexpected format at {uname}, after #{j}, {key}")

    # Store the commands and arguments into a database file.
    conn = sqlite3.Connection(dbfile)
    try:
        cl=pandas.DataFrame(command_list,columns=['i','cmd','uname']).set_index('i')
        al=pandas.DataFrame(arg_list,columns=['i','j','key','val']).set_index(['i','j'])

        cl.to_sql("command_list",conn,if_exists='replace')
        al.to_sql("arg_list",conn,if_exists='replace')
    finally:
        conn.close()

# Add a bunch of relations between components to help identify links between them
# as well as to filter out refrigeration components and give style info for the drawing.
def setup_db(dbfile,sqlfile="sql_scripts.txt"):
    with open(sqlfile) as f:
        script=f.read()
    conn = sqlite3.Connection(dbfile)
    try:
        conn.executescript(script)
    finally:
        conn.close()

# Version 1:
# Remove quotes and surrounding parenthesis from the right-hand side value for a keyword line.
def clean_value(val):
    m=re.match(r'"(?P<quoted>.*?)"|\(\s*(?P<list>.+?)\s*\)|(?P<unquoted>[^\(\)]*)',val)
    if m.lastgroup == "list":
        return val
    else:
        return m.group(m.lastgroup)

# Version 2:
# Remove quotes from the right-hand side value for a keyword line.
# If the value is a (list,of,items), then return a list (always flat).
# This is quick and dirty, don't rely on it.
def clean_value2(val,verbose=False):
    pattern0=r'\s*"(?P<quoted>.*?)"\s*|\s*\(\s*(?P<list>.+)\s*\)|\s*(?P<unquoted>[^"\s\(\)\,]*)'
    pattern1=r'\s*"(?P<quoted>.*?)"\s*,?|\s*(?P<unquoted>[^"\s\(\)\,]+)\s*,?'
    m0=re.match(pattern0,val)
    s0=m0.group(m0.lastgroup)
    if m0.lastgroup == "list":
        if verbose:
            print("IT's a list!",s0)
        result=[]
        tail=s0
        for m1 in re.finditer(pattern1,s0):
            s1 = m1.group(m1.lastgroup)
            if verbose:
                print(f"{m1.lastgroup}, '{s1}'")
            result.append(s1)
            tail=s0[m1.end():]
        if verbose:
            print(f"tail = '{tail}' ({len(tail)})")
        if len(tail) > 0:
            result.append(clean_value2(tail))
        return result
    else:
        return s0

# Given the database file of commands and keyword arguments,
# extract the links between refrigeration components
# (e.g. ATTACH-TO, etc)
def get_edges(dbfile):
    edge_list=[]
    conn = sqlite3.Connection(dbfile)
    try:
        edges=pandas.read_sql("""select "uname", "val" FROM "arg_extended_refg_network";""",conn)
    finally:
        conn.close()
    # The keyword values in the database are raw text from the input file.
    # So we should also clean up surrounding quotes and lists.
    #edges['val']=edges['val'].apply(clean_value)
    for row in edges.itertuples():
        cv = clean_value2(row.val)
        if type(cv) == list:
            edge_list.extend([(row.uname, val) for val in cv])
        else:
            edge_list.append((row.uname, cv))
    return edge_list

# Just the list of refrigeration components, plus style info for the drawing.
def get_nodes(dbfile):
    conn = sqlite3.Connection(dbfile)
    try:
        nodes = pandas.read_sql("""select "cmd", "uname", "shape", "color" FROM "refg_nodes";""",conn)
    finally:
        conn.close()
    return nodes

# Style info for the drawing, used only to draw the legend.
def get_cmd2shape(dbfile):
    conn = sqlite3.Connection(dbfile)
    try:
        cmd2shape = pandas.read_sql("""select * FROM "cmd2shape";""",conn)
    finally:
        conn.close()
    return cmd2shape

# Helper function for drawing legend in two columns.
def taker(iterable, n):
    while True:
        result = list(islice(iterable, n))
        if len(result) > 0:
            yield result
        else:
            break


# Draws the refrigeration system using graphviz and returns the graph.
#
# Optionally, render to a file named "basename.graphformat".
#
# Be advised of a weird bug with Dot engine. When rendering to PDF (only),
# the labels don't show on a node with gradient color that is within a subgraph/cluster.
# This is not a problem when rendering to SVG.
# So, for best results with PDF, avoid gradient colors like color1:color2.
def make_graph(basename,nodes,edges,cmd2shape,pdf_file,graphformat="pdf"):  
    from graphviz import Digraph

    dot = Digraph(comment=basename,engine='dot',format=graphformat)

    for row in nodes.itertuples():
        dot.node(row.uname, shape=row.shape, fillcolor=row.color, style="filled")
    for uname,val in edges:
        dot.edge(uname,val)
    
    # Draw a legend showing the shape and color for each command.
    # Use cluster subgraph to break them into multiple columns.
    # I had 18 types of items to draw, so put 9 in each column. 
    for i,group in enumerate(taker(cmd2shape.itertuples(),9)):
        with dot.subgraph(name=f'cluster_{i}') as c:
            # For each item after the first, connect it to the previous item
            # so that the stack vertically.
            lastname=None
            lastlabel=None
            
            for row in group:
                name=f"legend_{row.Index}"
                if row.type:
                    # E.g. condenser has subtypes air-cooled, evaporative, and cascaded
                    label=rf"{row.cmd}\n{row.type}"
                else:
                    # We don't distinguish subtypes for this kind of component.
                    label=f"{row.cmd}"
                if graphformat == "pdf":
                    # Don't use gradient colors
                    color = row.color.split(":")[0] 
                else:
                    color = row.color
                c.node(name, label=label, shape=row.shape, fillcolor=color, style="filled", fontcolor="black")
                if lastname:
                    c.edge(name, lastname, color="transparent")
                lastname=name
                lastlabel=label
    
    if graphformat:
        dot.render(pdf_file)
    return dot

# Extract a "wide" DataFrame for all components of a given kind,
# with each row representing a component, and each column showing a keyword.
def datatable_for_command(dbfile,cmd):
    conn = sqlite3.Connection(dbfile)
    try:
        df2=pandas.read_sql("""select "uname", "key", "val" FROM "arg_extended" where "cmd" == ?;""",
                            conn,
                            index_col=['uname','key'],
                            params=(cmd,))
    finally:
        conn.close()
    df3=df2.unstack()
    df3.columns=df3.columns.droplevel(0)
    return df3

# Store the DataFrame into an Excel file for later inspection by the user.
def save_table_to_excel(dbfile,cmd,xlfile):
    df3=datatable_for_command(dbfile,cmd)
    dt=df3.dtypes
    for c in df3:
        try:
            df3[c].astype(float)
            dt[c] = 'float'
        except:
            pass
    # Append to the file (don't overwrite it).
    with pandas.ExcelWriter(xlfile, engine="openpyxl", mode='a') as writer:
        df3.astype(dt).to_excel(writer,sheet_name=cmd,freeze_panes=(1,1))


In [ ]:
import glob
inpfiles=glob.glob(r"examples\*.inp")
inpfiles

In [ ]:
main('examples\\example1.inp')


In [ ]:
for inpfile in inpfiles:
    print(inpfile)
    dot=main(inpfile,"pdf")

In [ ]:
dot

Debugging

In [ ]:
inpfile='examples\\example1.inp'
basename,_=os.path.splitext(os.path.basename(inpfile))
dbfile=f"output/{basename}.db"
edges = get_edges(dbfile)
nodes = get_nodes(dbfile)

In [ ]:
for n in nodes.itertuples():
    if n.uname != clean_value(n.uname):
        print(n)

for uname,val in edges:
    if '"' in uname or '(' in val or ',' in val:
        print(uname,val)
        break

In [ ]:
pandas.DataFrame(edges)

In [ ]:
clean_value2(' ("thing1 thing2", "t")',True)

In [ ]:
pandas.options.display.max_rows=200

In [ ]:
edges['val'].apply(clean_value)

In [ ]:
edges['val']=edges['val'].apply(clean_value)

In [ ]:
from graphviz import Digraph
dot = Digraph(engine='dot',format='pdf',colorscheme=)

In [ ]:
cmd2shape=get_cmd2shape(dbfile)
cmd2shape

In [ ]:
lastrow=None
for row in cmd2shape.itertuples():
    if row.type:
        label=rf"{row.cmd}\n{row.type}"
    else:
        label=f"{row.cmd}"
    print(label)
    if lastrow:
        print(f"-> {lastrow.cmd}")
    
    lastrow=row
    print()

In [ ]:
# Helper function to group legend items into multiple columns

# (1)
# In this version the final group yields None elements several times at the end.
# This is a purely iterator-based solution.
# https://docs.python.org/3/library/itertools.html#itertools-recipes
def grouper(iterable, n, fillvalue=None):
    from itertools import zip_longest
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)

# (2)
# Another purely iterator-based solution.
# With this version, the final group never yields None.
# However, it may yield an empty final group if n evenly divides the number of items.
class grouper2:
    from itertools import islice
    def __init__(self, iterable, n):
        self.iterable=iterable
        self.n=n
        self.done = False
    def __iter__(self):
        return self
    def __next__(self):
        if self.done:
            raise StopIteration
        return self.mynext()
    def mynext(self):
        for i in range(self.n):
            try:
                yield next(self.iterable)
            except StopIteration:
                self.done = True

# (3)
# This version puts the items for each group into a list.
# It never returns an empty group or None items.
def taker(iterable, n):
    while True:
        result = list(islice(iterable, n))
        if len(result) > 0:
            yield result
        else:
            break

In [ ]:
for i,group in enumerate(grouper(enumerate(cmd2shape.itertuples()),10)):
    print(f"Block {i}")
    lastrow=None
    for j,row in group:
        print(j)
        print(row.cmd)
    print(f"Finished the group")
    print()


In [ ]:
for i,group in enumerate(grouper2(enumerate(cmd2shape.itertuples()),9)):
    print(f"Block {i}")
    lastrow=None
    for j,row in group:
        print(j)
        print(row.cmd)
    print(f"Finished the group")
    print()


In [ ]:
for i,group in enumerate(taker(enumerate(cmd2shape.itertuples()),9)):
    print(f"Block {i}")
    lastrow=None
    for j,row in group:
        print(j)
        print(row.cmd)
    print(f"Finished the group")
    print()


In [ ]:
row

In [ ]:
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPDF, renderPM

In [ ]:
import cairosvg

In [ ]:
# Needs more work to recognize nested lists
clean_value2('(list, (of lists) and "items" (etc and more) )', True)

In [ ]:
clean_value2('( (etc and more) )')